# EDA Rápido Antes de la Limpieza

#### 1. **Estructura y Dimensiones**
- Número de filas y columnas (`datos.shape`)
- Tipos de datos de cada columna (`datos.dtypes`)
- Primeras y últimas filas (`datos.head()`, `datos.tail()`)

#### 2. **Estadísticas Generales**
- Estadísticas descriptivas de variables numéricas (`datos.describe()`)
- Estadísticas de variables categóricas (`datos.describe(include='object')`)

#### 3. **Valores Nulos**
- Conteo de valores nulos por columna (`datos.isnull().sum()`)
- Porcentaje de nulos por columna

#### 4. **Duplicados**
- Número de filas duplicadas (`datos.duplicated().sum()`)

#### 5. **Valores Únicos**
- Número de valores únicos por columna (`datos.nunique()`)

#### 6. **Distribución de Variables**
- Histogramas y boxplots para variables numéricas
- Conteo de frecuencias para variables categóricas

#### 7. **Outliers**
- Detección visual con boxplots
- Revisión de valores extremos en estadísticas descriptivas

#### 8. **Relaciones entre Variables**
- Correlaciones entre variables numéricas (`datos.corr()`)
- Tablas cruzadas para variables categóricas

In [ ]:
# Importar Librerías necesarias
import pandas as pd # Data manipulation
import numpy as np # Numerical operations
import matplotlib.pyplot as plt # Data visualization
import seaborn as sns # Data visualization
import plotly.express as px # Interactive data visualization
import sys
sys.path.append('..')  # Ajusta el path si es necesario

from main import cargar_datos

In [ ]:
# Cargar el dataset
datos = cargar_datos()

#### 1. **Estructura y Dimensiones**
- Número de filas y columnas (`datos.shape`)
- Tipos de datos de cada columna (`datos.dtypes`)
- Primeras y últimas filas (`datos.head()`, `datos.tail()`)

In [ ]:
# Números de filas y columnas
print(f"El dataset contiene {datos.shape[0]} filas y {datos.shape[1]} columnas.")

In [ ]:
# Tipos de datos de cada columna
print("Tipos de datos de cada columna:")
datos.dtypes

In [ ]:
# Primeras filas
datos.head()

In [ ]:
# Ultimas filas
datos.tail()

**Interpretación:**

* *Hay 250 filas y 11 columnas*
* *Hay un tipo de dato que debemos corregir, es el de `date` debe ser de tipo fecha*

In [ ]:
# Cambio de tipo de dato de la columna 'Date' a datetime
datos['Date'] = pd.to_datetime(datos['Date'], format='%d-%m-%y')

#### 2. **Estadísticas Generales**
- Estadísticas descriptivas de variables numéricas (`datos.describe()`)
- Estadísticas de variables categóricas (`datos.describe(include='object')`)

In [ ]:
# Estadísticas descriptivas de variables numéricas
datos.describe()

In [ ]:
# Estadísticas descriptivas de variables categóricass
datos.describe(include=['object'])

#### 3. **Valores Nulos**
- Conteo de valores nulos por columna (`datos.isnull().sum()`)
- Porcentaje de nulos por columna

In [ ]:
# Conteo de valores nulos
print("Conteo de valores nulos por columna:")
datos.isnull().sum()

In [ ]:
# Porcemtaje de valores nulos por columna
porcentaje_nulos = datos.isnull().mean() * 100
print("Porcentaje de valores nulos por columna:")
print(porcentaje_nulos)

**Interpretación:**

* No hay ***nulos*** en el dataset

#### 4. **Duplicados**
- Número de filas duplicadas (`datos.duplicated().sum()`)

In [ ]:
# Numero de duplicados
print("Número de filas duplicadas:")
print(datos.duplicated().sum())

**Interpretación:**

* No hay ***duplicados*** en el dataset

#### 5. **Valores Únicos**
- Número de valores únicos por columna (`datos.nunique()`)

In [ ]:
# número de valores únicos por columna
print("Número de valores únicos por columna:")
datos.nunique()

#### 6. **Distribución de Variables**
- Histogramas y boxplots para variables numéricas
- Conteo de frecuencias para variables categóricas

In [ ]:
# Histogrmas y boxplots para variables numéricas
numericas = datos.select_dtypes(include=[np.number]).columns.tolist()
for col in numericas:
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(datos[col], kde=True)
    plt.title(f'Histograma de {col}')
    
    plt.subplot(1, 2, 2)
    sns.boxplot(x=datos[col])
    plt.title(f'Boxplot de {col}')
    
    plt.tight_layout()
    plt.show()

En tu análisis:

* **Price**: Distribución sesgada a la derecha (muchos precios bajos y pocos muy altos), típica en ventas.
* **Quantity**: Distribución casi uniforme (las cantidades de 1 a 5 se venden con frecuencias similares).
* **Total Sales**: Distribución sesgada a la derecha (muchas ventas bajas y pocas muy altas).

In [ ]:
# Conteo de frecuencias para viariables categóricas excepto Order ID
categoriales = datos.select_dtypes(include=['object']).columns.tolist()
categoriales.remove('Order ID')  # Excluir 'Order ID'
for col in categoriales:
    plt.figure(figsize=(10, 5))
    sns.countplot(y=datos[col], order=datos[col].value_counts().index)
    plt.title(f'Conteo de frecuencias de {col}')
    plt.show()

#### 7. **Outliers**
- Detección visual con boxplots
- Revisión de valores extremos en estadísticas descriptivas

In [ ]:
# Detección visual de outliers con boxplots usando plotly
for col in numericas:
    fig = px.box(datos, y=col, title=f'Boxplot interactivo de {col}')
    fig.show()

In [ ]:
# Detectar outliers usando el método IQR
def detectar_outliers_iqr(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    return df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]
for col in numericas:
    outliers = detectar_outliers_iqr(datos, col)
    print(f"Outliers en {col}:")
    print(outliers[[col]])
    print("\n")
    

In [ ]:
# Ver tabla con los ouliers hallados
outliers_tabla = pd.DataFrame()
for col in numericas:
    outliers = detectar_outliers_iqr(datos, col)
    outliers_tabla[col] = outliers[col]
outliers_tabla = outliers_tabla.dropna(how='all')  # Eliminar columnas sin outliers
print("Tabla de outliers detectados:")
outliers_tabla

**Interpretación:**

* Muestra una tabla donde se ve que hay en total 11 outliers en la columna `Total Sales`, lo cual lo vamos a verificar si son outliers en la Limpieza de datos

#### 8. **Relaciones entre Variables**
- Correlaciones entre variables numéricas (`datos.corr()`)
- Tablas cruzadas para variables categóricas

In [ ]:
# Headmap usando plotly solo con columnas numéricas relevantes
columnas_numericas = datos.select_dtypes(include=[np.number]).columns
fig = px.imshow(datos[columnas_numericas].corr(), text_auto=True, aspect="auto", title="Mapa de calor de correlaciones")
fig.update_layout(width=700, height=500)
fig.show()

In [ ]:
# Correlaciones entre variables numéricas
correlaciones = datos[numericas].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de calor de correlaciones entre variables numéricas')


***

**El mapa de calor muestra:**

* **Price y Total Sales**: correlación fuerte positiva (**0.85**), lo que indica que a mayor precio, mayores ventas totales (posiblemente por un efecto de ticket promedio).
* **Quantity y Total Sales**: correlación positiva moderada (**0.33**), sugiere que más cantidad vendida sí incrementa ventas totales, pero no tanto como el precio.
* **Price y Quantity**: correlación prácticamente nula (**-0.01**), indicando independencia entre precio y cantidad vendida.

Esto significa que el precio influye más en las ventas totales que la cantidad.

***

**En palabras simples:**

* **Precio y ventas totales** (0.85): Cuando sube el precio, también suben las ventas totales.
* **Cantidad y ventas totales** (0.33): Vender más unidades ayuda, pero no tanto como subir el precio.
* **Precio y cantidad** (-0.01): El precio no afecta cuántas unidades se venden.

***

En el análisis, esto dice que:

* Para mover más las **ventas totales**, es más efectivo ajustar el **precio** que la cantidad vendida.
* La **cantidad** sí suma, pero su impacto es menor.
* **Precio y cantidad** son independientes, así que puedes optimizar cada uno por separado.


In [ ]:
# Tablas cruzadas para variables categóricas

tabla_cruzada = pd.crosstab(index=datos['Product'], columns='count')
print(f"Tabla cruzada para {col}:")
tabla_cruzada
    

In [ ]:
tabla_cruzada = pd.crosstab(index=datos['Category'], columns='count')
print(f"Tabla cruzada para {col}:")
tabla_cruzada

In [ ]:
tabla_cruzada = pd.crosstab(index=datos['Customer Name'], columns='count')
print(f"Tabla cruzada para {col}:")
tabla_cruzada

In [ ]:
tabla_cruzada = pd.crosstab(index=datos['Customer Location'], columns='count')
print(f"Tabla cruzada para {col}:")
tabla_cruzada

In [ ]:
tabla_cruzada = pd.crosstab(index=datos['Payment Method'], columns='count')
print(f"Tabla cruzada para {col}:")
tabla_cruzada

In [ ]:
tabla_cruzada = pd.crosstab(index=datos['Status'], columns='count')
print(f"Tabla cruzada para {col}:")
tabla_cruzada